## **NGC 6302 - Nebulosa de la Mariposa**

Base de código para el cálculo de: <br>
- centroide (MÁSCARA PARA NÚCLEO)
- eje principal y eje de envergadura (EJE PRINCIPAL DE LA NEBULOSA)
- transformación XY - V2V3 - NE (TRANSFORMACIÓN DE COORDENADAS)
- estrellas brillantes (DETECCIÓN DE ESTRELLAS BRILLANTES)
- distancia del centroide a las estrellas más brillants (DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES)
- visualización con todo lo anterior, orientación, retícula, ejes... (VISUALIZACIÓN)


In [ ]:
# Librerías necesarias
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.colors import LogNorm
from astropy.wcs import WCS
import math
from astropy import units as u
from scipy.ndimage import label, center_of_mass
from skimage.measure import regionprops
from matplotlib.patches import Circle
from skimage.feature import peak_local_max
from sklearn.decomposition import PCA

# Cargar la fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar los datos de la imagen
F110W = fits.open('NGC6302/ie3o02050_drz.fits')  # FITS file
F110W_h = F110W[0].header
F110W_d = fits.getdata('NGC6302/ie3o02050_drz.fits')
F110W_d = np.nan_to_num(F110W_d)  # Reemplazar NaN por 0

# Obtener dimensiones
height, width = F110W_d.shape
print(f"Píxeles de la imagen: {F110W_d.shape}")

# Obtener WCS
wcs = WCS(F110W[1].header, naxis=2)

###################### PASO DE AR - h m s Y DEC º ' " ######################
# Obtener AR y DEC de la cabecera
AR = F110W_h['RA_TARG']
DEC = F110W_h['DEC_TARG']
# Función para convertir grados a horas, minutos, segundos (1h son 15º)
def grados_a_hms(ar_deg):
    horas = int(ar_deg / 15)  # Convertir grados a horas
    minutos = int((ar_deg / 15 - horas) * 60)  # Convertir la parte fraccionaria de horas a minutos
    segundos = ((ar_deg / 15 - horas) * 60 - minutos) * 60  # Convertir la parte fraccionaria de minutos a segundos
    return f"{horas}h {minutos}m {segundos:.2f}s"
# Convertir AR a formato de horas, minutos y segundos
AR_hms = grados_a_hms(AR)

# Función para convertir grados a grados, minutos y segundos
def grados_a_gms(deg):
    grados = int(deg)  # Parte entera de los grados
    minutos = int(abs(deg - grados) * 60)  # Convertir la parte decimal de grados a minutos
    segundos = (abs(deg - grados) * 60 - minutos) * 60  # Convertir la parte decimal de minutos a segundos
    return f"{grados}° {minutos}' {segundos:.2f}'' "

# Convertir DEC a formato de grados, minutos y segundos
DEC_gms = grados_a_gms(DEC)

##################################################################

#####################################   MÁSCARA PARA NÚCLEO   #####################################

#  Definir el radio de interés en píxeles para el núcleo
R_nucleo = 450  # Reducimos el radio para centrarnos más en la zona brillante

#  Obtener el centro de la imagen
center_x, center_y = height // 2, width // 2

#  Crear una máscara circular para limitar la búsqueda del núcleo
Y, X = np.ogrid[:height, :width]
mask_circle_nucleo = (X - center_x) ** 2 + (Y - center_y) ** 2 <= R_nucleo ** 2  # Ecuación del círculo

#  Aplicar la máscara circular a la imagen original
masked_F110W = np.where(mask_circle_nucleo, F110W_d, 0)

#  Aplicar un umbral de brillo SOLO dentro del círculo del núcleo
umbral_nucleo = np.percentile(masked_F110W[mask_circle_nucleo], 75)  # Subimos el umbral para mejor precisión
mask_nucleo = masked_F110W > umbral_nucleo

#  Encontrar las regiones conectadas del núcleo
labeled, num_features = label(mask_nucleo)
region_sizes = np.bincount(labeled.ravel())[1:]  # Contar tamaños de regiones
largest_region_idx = np.argmax(region_sizes) + 1  # Índice de la región más grande
mask_nucleo = labeled == largest_region_idx  # Filtrar solo la región central

#  Obtener el centroide ponderado (núcleo de la nebulosa)
centroid = center_of_mass(masked_F110W * mask_nucleo)
centroid = (int(centroid[0]), int(centroid[1]))  # Convertir a enteros
print('---------------------------------------------------------------------------')
print(f"Centro refinado de NGC6302: {centroid}")

# Convertir a coordenadas celestes
RA_centroid, DEC_centroid = wcs.all_pix2world([[centroid[1], centroid[0]]], 1)[0]
RA_centroid_hms = grados_a_hms(RA_centroid)
DEC_centroid_gms = grados_a_gms(DEC_centroid)
print(f"Centro refinado de NGC 6302 en coordendas (α,δ): ({RA_centroid_hms}, {DEC_centroid_gms}) ")
print('----------------------------------------------------------------------------')

#####################################   EJE PRINCIPAL DE LA NEBULOSA   #####################################
# Definir un radio de interés en píxeles para delimitar la envergadura
R_eje = 500  # Ajustar este valor para cubrir solo la nebulosa más brillante

# Crear una máscara circular para limitar la búsqueda de la envergadura
mask_circle_eje = (X - centroid[1]) ** 2 + (Y - centroid[0]) ** 2 <= R_eje ** 2

# Aplicar un umbral de brillo más alto para detectar SOLO la nebulosa dentro de la máscara circular
umbral_eje = np.percentile(F110W_d[mask_circle_eje], 75)  # Ajuste fino para delimitar la nebulosa
nebula_mask = np.where(mask_circle_eje, F110W_d, 0) > umbral_eje  # Aplicar máscara circular

# Determinar los extremos de la línea de envergadura dentro de la máscara de brillo
row = centroid[0]  # Tomamos la fila correspondiente al núcleo detectado
valid_pixels = np.where(nebula_mask[row, :])[0]  # Detectar los píxeles dentro de la nebulosa en esa fila

# Establecer la envergadura SIMÉTRICA respecto al núcleo
max_extent = min(centroid[1] - valid_pixels[0], valid_pixels[-1] - centroid[1])  # Simetría
x_min = int(centroid[1] - max_extent)
x_max = int(centroid[1] + max_extent)

# Convertir a coordenadas ecuatoriales para el plot
RA_start, DEC_start = wcs.all_pix2world([[x_min, row]], 1)[0]
RA_end, DEC_end = wcs.all_pix2world([[x_max, row]], 1)[0]

# -----------------------------------------------------------------
# PCA sobre la máscara para obtener dirección principal
coords_eje = np.column_stack(np.where(nebula_mask))
pca_eje = PCA(n_components=2)
pca_eje.fit(coords_eje)
dir_eje = pca_eje.components_[0]  # dirección principal

# Dirección perpendicular (envergadura)
dir_perp = np.array([-dir_pca[1], dir_eje[0]])

# Proyección en dirección perpendicular
projections_eje = np.dot(coords_eje - np.array([centroid[0], centroid[1]]), dir_eje)
max_extent_eje = np.max(np.abs(projections_eje))  # en píxeles

# Coordenadas de la envergadura centrada en el núcleo
cx, cy = centroid[1], centroid[0]
x1_eje = cx - max_extent_eje * dir_perp[1]
y1_eje = cy - max_extent_eje * dir_perp[0]
x2_eje = cx + max_extent_eje * dir_perp[1]
y2_eje = cy + max_extent_eje * dir_perp[0]

#---------------------------------------------------

# PCA sobre la máscara para obtener dirección principal
coords_enverg = np.column_stack(np.where(nebula_mask))
pca_enverg = PCA(n_components=2)
pca_enverg.fit(coords_enverg)
dir_env = pca_enverg.components_[0]  # dirección principal

# Proyección en dirección perpendicular
projections = np.dot(coords_enverg - np.array([centroid[0], centroid[1]]), dir_env)
max_extent_env = np.max(np.abs(projections))  # en píxeles

# Coordenadas de la envergadura centrada en el núcleo
cx, cy = centroid[1], centroid[0]
x1_env = cx - max_extent_env * dir_env[1]
y1_env = cy - max_extent_env * dir_env[0]
x2_env = cx + max_extent_env * dir_env[1]
y2_env = cy + max_extent_env * dir_env[0]

#####################################   TRANSFORMACIÓN DE COORDENADAS   #####################################
# Datos de entrada
Xr = 512.0
Yr = 512.0
V2r = 1.1794  # en arcosegundos
V3r = -0.4119 # en arcosegundos
Sx = 0.135437
Sy = 0.120944
beta_x = -45.1585  # en grados
beta_y = 44.6677   # en grados
X = 472.0
Y = 553.0
PA_V3 = 280.032104  # Ángulo PA_V3 en grados

# Conversión de grados a radianes, para poder trabajar con ellos
beta_x_rad = math.radians(beta_x)
beta_y_rad = math.radians(beta_y)

# Convertir PA_V3 a radianes
PA_V3_rad = math.radians(PA_V3)

# Cálculo de V2 arcosegundos
V2 = V2r + Sx * math.sin(beta_x_rad) * (X - Xr) + Sy * math.sin(beta_y_rad) * (Y - Yr)

# Cálculo de V3 arcosegundos
V3 = V3r + Sx * math.cos(beta_x_rad) * (X - Xr) + Sy * math.cos(beta_y_rad) * (Y - Yr)

# Rotación de las coordenadas V2 y V3 al sistema Norte y Este
N = V3 * math.cos(PA_V3_rad) - V2 * math.sin(PA_V3_rad)  # Coordenada Norte relativa
E = V3 * math.sin(PA_V3_rad) + V2 * math.cos(PA_V3_rad)  # Coordenada Este relativa

print('----------------------------------------------------------------------------')
print('')

#####################################   DETECCIÓN DE ESTRELLAS BRILLANTES   #####################################
# Paso 1: Normalización (si no la tienes ya hecha)
img_norm = (F110W_d - np.min(F110W_d)) / (np.max(F110W_d) - np.min(F110W_d))

# Paso 2: Detección de picos locales (estrellas brillantes)
coords_estrellas = peak_local_max(img_norm, min_distance=5, threshold_abs=0.4)  # Puedes ajustar el umbral

# Mensaje informativo
print(f"Se han detectado y marcado las {len(coords_estrellas)} estrellas más brillantes.")

####################################   DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES  #####################################

# Distancia en años luz
d_ly = 3400 # información sacada de internet
ly_to_pc = 0.3066
d_pc = d_ly * ly_to_pc  

# Escala de píxel (arcsec/pix)
pixel_scales = np.abs(wcs.pixel_scale_matrix.diagonal()) * 3600
arcsec_per_pix = np.mean(pixel_scales)

# Calcular distancias en píxeles desde el centroide
pix_distances = [np.linalg.norm(np.array([y, x]) - np.array(centroid)) for y, x in coords_estrellas]

# Pasar a arcsec
arcsec_distances = np.array(pix_distances) * arcsec_per_pix

# Pasar a radianes
theta_rad = (arcsec_distances / 3600) * (np.pi / 180)

# Distancias físicas en parsec
pc_distances = theta_rad * d_pc

# Mostrar 5 resultados más representativos (puedes ajustar el número)
print('\nDistancias desde el núcleo a las estrellas más brillantes:')
for i, (p, a, pc) in enumerate(zip(pix_distances[:5], arcsec_distances[:5], pc_distances[:5]), 1):
    print(f"Estrella {i}: {p:.1f} px ≈ {a:.2f}\" ≈ {pc:.2f} pc")

#####################################   VISUALIZACIÓN   #####################################

# Crear la figura sin distorsiones
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=wcs)  ## Trabajar en 2D

#  Ajuste del colormap y el contraste
im = ax.imshow(F110W_d, cmap='inferno', norm=LogNorm(vmin=1e1, vmax=1e4))  # Ajustado para más contraste
ax.set_title('NGC 6302 - Nebulosa Planetaria de Mariposa (F110W)', fontproperties=tnr_font, fontsize=16, pad=15)
ax.set_xlabel('α', fontproperties=tnr_font, fontsize=12)
ax.set_ylabel('δ', fontproperties=tnr_font, fontsize=12)
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Marcar el centro refinado en la imagen
ax.plot(centroid[1], centroid[0], '+', color='white', markersize=7, label="Centro Refinado")

# Dibujar círculos estrellas más brillantes y líneas desde el núcleo
for i, (y, x) in enumerate(coords_estrellas[:5]):  # solo las 5 más cercanas por claridad
    ax.add_patch(Circle((x, y), radius=10, edgecolor='lime', facecolor='none', linewidth=1.6))
    ax.plot([centroid[1], x], [centroid[0], y], color='orange', linestyle='--', linewidth=1.2)
    ax.text(x + 12, y, f'{i+1}', color='white', fontsize=13, fontproperties=tnr_font,
            ha='left', va='center', bbox=dict(facecolor='black', alpha=0.5, pad=1.5))


# Dibujar eje
ax.plot([x1_env, x2_env], [y1_env, y2_env], color='cyan', linewidth=2, label="Envergadura")
ax.text(cx + 250, cy + 240, 'Envergadura', color='cyan',
        fontsize=10, fontproperties=tnr_font)

# Dibujar envergadura
ax.plot([x1_eje, x2_eje], [y1_eje, y2_eje], color='red', linewidth=2, label="Eje principal")
ax.text(cx - 170, cy + 320, 'Eje principal', color='red',
        fontsize=10, fontproperties=tnr_font)

# Añadir anotación para AR y DEC
text_str = f"Coordenadas del núcleo\nα: {grados_a_hms(RA_centroid)}\nδ: {grados_a_gms(DEC_centroid)}"
ax.text(0.25, 0.95, text_str, transform=ax.transAxes, color='white', fontsize=10,
        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# Añadir anotación para N y E
#text_str = f"Nc: {N:.6f} (arcsec)\nEc: {E:.6f} (arcsec)"
#ax.text(0.95, 0.855, text_str, transform=ax.transAxes, color='white', fontsize=10,
#        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))


# --------------------------
# Añadir Norte y Este en la retícula correcta
# Punto en la esquina inferior derecha para la orientación
x, y = 960, 170.5
ra_dec_corner = wcs.pixel_to_world(x, y)

# Desplazamientos para las flechas
delta_ra = (0.002 * u.deg)
delta_dec = (0.002 * u.deg)

# Calcular coordenadas de Norte y Este
ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)

# Convertir de vuelta a píxeles
norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
este_x, este_y = wcs.world_to_pixel(ra_dec_este)

# Dibujar flechas en la dirección correcta
ax.arrow(x, y, norte_x - x, norte_y - y, 
         color='white', head_width=20, head_length=25, label="Norte")
ax.arrow(x, y, este_x - x, este_y - y, 
         color='white', head_width=20, head_length=25, label="Este")

# Etiquetas
ax.text(norte_x + 25, norte_y + 15, 'N', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')
ax.text(este_x, este_y - 40, 'E', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')


# Añadir la barra de color
cbar = plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Brillo', fontproperties=tnr_font, fontsize=12)
cbar.ax.tick_params(labelsize=10)
for label in cbar.ax.get_yticklabels():
    label.set_fontproperties(tnr_font)


# Configurar la retícula correctamente
ax.grid(color='white', linestyle='--', linewidth=0.5)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Ajuste del espacio

plt.show()
F110W.close()


### Base de código para realizar la composición RGB

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
import matplotlib.font_manager as fm

# Fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar imágenes y WCS (asegurando abrir todos desde la misma carpeta)
F164N_hdu = fits.open('NGC6302/ie3o02010_drz.fits')  # Hα
F128N_hdu = fits.open('NGC6302/ie3o02030_drz.fits')  # [O III]
F130N_hdu = fits.open('NGC6302/ie3o02040_drz.fits')  # He II

F164N = np.nan_to_num(F164N_hdu[1].data)
F128N = np.nan_to_num(F128N_hdu[1].data)
F130N = np.nan_to_num(F130N_hdu[1].data)
wcs = WCS(F164N_hdu[1].header)  # Usamos WCS de la primera imagen, por ejemplo

# Igualar tamaños
min_shape = (min(F164N.shape[0], F128N.shape[0], F130N.shape[0]), 
             min(F164N.shape[1], F128N.shape[1], F130N.shape[1]))

F164N_crop = F164N[:min_shape[0], :min_shape[1]]
F128N_crop = F128N[:min_shape[0], :min_shape[1]]
F130N_crop = F130N[:min_shape[0], :min_shape[1]]

# Normalización individual
def normalize_img(img):
    img_clip = np.clip(img, np.percentile(img[img > 0], 5), np.percentile(img[img > 0], 99))
    norm = (img_clip - img_clip.min()) / (img_clip.max() - img_clip.min())
    return norm

R = normalize_img(F164N_crop)
G = normalize_img(F128N_crop)
B = normalize_img(F130N_crop)

# Crear imagen RGB
rgb_image = np.zeros((R.shape[0], R.shape[1], 3))
rgb_image[:, :, 0] = R
rgb_image[:, :, 1] = G
rgb_image[:, :, 2] = B

# Función para flechas N y E
def N_E_arrows(ax, wcs):
    x, y = 950, 170
    ra_dec_corner = wcs.pixel_to_world(x, y)
    delta_ra = 0.002 * u.deg
    delta_dec = 0.002 * u.deg
    ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
    ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)
    norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
    este_x, este_y = wcs.world_to_pixel(ra_dec_este)

    ax.arrow(x, y, norte_x - x, norte_y - y, color='red', head_width=20, head_length=25)
    ax.arrow(x, y, este_x - x, este_y - y, color='red', head_width=20, head_length=25)
    ax.text(norte_x + 10, norte_y + 30, 'N', color='red', fontsize=15, fontproperties=tnr_font, ha='center')
    ax.text(este_x - 10, este_y - 40, 'E', color='red', fontsize=15, fontproperties=tnr_font, ha='center')

# Mostrar subplots con cada filtro y la combinación
fig = plt.figure(figsize=(25, 20))
plt.subplots_adjust(wspace=0.4)  # Aumenta la separación horizontal

# Hα
ax1 = fig.add_subplot(2, 3, 1, projection=wcs)
ax1.imshow(R, cmap='Reds')
ax1.set_title("F164N ([Fe II])", fontproperties=tnr_font, fontsize=24)
ax1.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax1, wcs)
ax1.set_xlabel('α', fontproperties=tnr_font, fontsize=18)
ax1.set_ylabel('δ', fontproperties=tnr_font, fontsize=18)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax1.coords[0].set_ticklabel(size=70, fontproperties=tnr_font, simplify=False)  # α (RA)
ax1.coords[1].set_ticklabel(size=70, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon1 = ax1.coords[0]  # Eje de Ascensión Recta (α)
lat1 = ax1.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon1.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat1.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon1.display_minor_ticks(True)
lat1.display_minor_ticks(True)
#############################################################################



# [O III]
ax2 = fig.add_subplot(2, 3, 2, projection=wcs)
ax2.imshow(G, cmap='Greens')
ax2.set_title("F128N (Hβ)", fontproperties=tnr_font, fontsize=24)
ax2.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax2, wcs)
ax2.set_xlabel('α', fontproperties=tnr_font, fontsize=18)
ax2.set_ylabel('δ', fontproperties=tnr_font, fontsize=18)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax2.coords[0].set_ticklabel(size=70, fontproperties=tnr_font, simplify=False)  # α (RA)
ax2.coords[1].set_ticklabel(size=70, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon2 = ax2.coords[0]  # Eje de Ascensión Recta (α)
lat2 = ax2.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon2.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat2.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon2.display_minor_ticks(True)
lat2.display_minor_ticks(True)
#############################################################################


# He II
ax3 = fig.add_subplot(2, 3, 3, projection=wcs)
ax3.imshow(B, cmap='Blues')
ax3.set_title("F130N (H molecular)", fontproperties=tnr_font, fontsize=24)
ax3.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax3, wcs)
ax3.set_xlabel('α', fontproperties=tnr_font, fontsize=18)
ax3.set_ylabel('δ', fontproperties=tnr_font, fontsize=18)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax3.coords[0].set_ticklabel(size=70, fontproperties=tnr_font, simplify=False)  # α (RA)
ax3.coords[1].set_ticklabel(size=70, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon3 = ax3.coords[0]  # Eje de Ascensión Recta (α)
lat3 = ax3.coords[1]  # Eje de Declinación (δ)


# Forzar la visualización completa de horas, minutos y segundos en α
lon3.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat3.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon3.display_minor_ticks(True)
lat3.display_minor_ticks(True)
#############################################################################


# RGB combinado
ax4 = fig.add_subplot(2, 3, 5, projection=wcs)
ax4.imshow(rgb_image)
ax4.set_title("NGC 6302 - Composición RGB (A)", fontproperties=tnr_font, fontsize=24)
ax4.grid(color='white', linestyle='--', linewidth=0.5)
N_E_arrows(ax4, wcs)
ax4.set_xlabel('α', fontproperties=tnr_font, fontsize=18)
ax4.set_ylabel('δ', fontproperties=tnr_font, fontsize=18)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax4.coords[0].set_ticklabel(size=70, fontproperties=tnr_font, simplify=False)  # α (RA)
ax4.coords[1].set_ticklabel(size=70, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon4 = ax4.coords[0]  # Eje de Ascensión Recta (α)
lat4 = ax4.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon4.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat4.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon4.display_minor_ticks(True)
lat4.display_minor_ticks(True)
#############################################################################


plt.tight_layout()
plt.show()


### Base de código para realizar los perfiles de brillo

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

# Cargar los filtros RGB de la composición A
F164N = np.nan_to_num(fits.getdata('NGC6302/ie3o02010_drz.fits'))  # R
F128N = np.nan_to_num(fits.getdata('NGC6302/ie3o02030_drz.fits'))  # G
F130N = np.nan_to_num(fits.getdata('NGC6302/ie3o02040_drz.fits'))  # B

# Igualar tamaño
min_shape = (min(F164N.shape[0], F128N.shape[0], F130N.shape[0]),
             min(F164N.shape[1], F128N.shape[1], F130N.shape[1]))
F164N = F164N[:min_shape[0], :min_shape[1]]
F128N = F128N[:min_shape[0], :min_shape[1]]
F130N = F130N[:min_shape[0], :min_shape[1]]

# Centro
center_x, center_y = min_shape[0] // 2, min_shape[1] // 2

# Suavizado opcional (mejor para perfiles suaves)
F164N_smooth = gaussian_filter(F164N, sigma=1.5)
F128N_smooth = gaussian_filter(F128N, sigma=1.5)
F130N_smooth = gaussian_filter(F130N, sigma=1.5)

# Calcular perfil radial
def perfil_radial(imagen, centro, r_max):
    y, x = np.indices(imagen.shape)
    r = np.sqrt((x - centro[1])**2 + (y - centro[0])**2).astype(int)
    perfil = np.zeros(r_max)
    for i in range(r_max):
        mask = (r == i)
        if np.sum(mask) > 0:
            perfil[i] = np.mean(imagen[mask])
    return perfil

# Radios hasta los que analizar (ajusta si quieres más lejos)
r_max = 100

# Calcular perfiles para R, G, B
perfil_R = perfil_radial(F164N_smooth, (center_x, center_y), r_max)
perfil_G = perfil_radial(F128N_smooth, (center_x, center_y), r_max)
perfil_B = perfil_radial(F130N_smooth, (center_x, center_y), r_max)

# --- PLOT ---
plt.figure(figsize=(8,6))
plt.plot(perfil_R, color='red', label='[Fe II] - F164N (R)')
plt.plot(perfil_G, color='green', label='H molecular - F128N (G)')
plt.plot(perfil_B, color='blue', label='continuo estelar - F130N (B)')
plt.title("Perfil Radial - Composición RGB A (NGC 6302)")
plt.xlabel("Distancia radial (pixeles)")
plt.ylabel("Brillo promedio")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## **M57 - Nebulosa del Anillo**

Base de código para el cálculo de: <br>
- centroide (MÁSCARA PARA NÚCLEO)
- eje mayor y menor (EJE PRINCIPAL DE LA NEBULOSA)
- transformación XY - V2V3 - NE (TRANSFORMACIÓN DE COORDENADAS)
- estrellas brillantes (DETECCIÓN DE ESTRELLAS BRILLANTES)
- distancia del centroide a las estrellas más brillants (DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES)
- visualización con todo lo anterior, orientación, retícula, ejes... (VISUALIZACIÓN)


In [ ]:
# Librerías necesarias
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.colors import LogNorm
from astropy.wcs import WCS
import math
from astropy import units as u
from astropy.coordinates import SkyCoord
from scipy.ndimage import label, center_of_mass
from skimage.measure import regionprops
from matplotlib.patches import Circle
from skimage.feature import peak_local_max

# Cargar la fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar los datos de la imagen
NGC6720 = fits.open('NGC6720/ibh601050_drz.fits')  # FITS file
NGC6720_h = NGC6720[0].header
M57 = fits.getdata('NGC6720/ibh601050_drz.fits')
M57 = np.nan_to_num(M57)  # Reemplazar NaN por 0

# Obtener dimensiones
height, width = M57.shape
print(f"Píxeles de la imagen: {M57.shape}")
print('----------------------------------------------------------------------------')
# Obtener WCS
wcs = WCS(NGC6720[1].header, naxis=2)

###################### PASO DE AR - h m s Y DEC º ' " ######################
# Función para convertir grados a horas, minutos, segundos (1h son 15º)
def grados_a_hms(ar_deg):
    horas = int(ar_deg / 15)  # Convertir grados a horas
    minutos = int((ar_deg / 15 - horas) * 60)  # Convertir la parte fraccionaria de horas a minutos
    segundos = ((ar_deg / 15 - horas) * 60 - minutos) * 60  # Convertir la parte fraccionaria de minutos a segundos
    return f"{horas}h {minutos}m {segundos:.2f}s"
# Convertir AR a formato de horas, minutos y segundos
#AR_hms = grados_a_hms(AR)

# Función para convertir grados a grados, minutos y segundos
def grados_a_gms(deg):
    grados = int(deg)  # Parte entera de los grados
    minutos = int(abs(deg - grados) * 60)  # Convertir la parte decimal de grados a minutos
    segundos = (abs(deg - grados) * 60 - minutos) * 60  # Convertir la parte decimal de minutos a segundos
    return f"{grados}° {minutos}' {segundos:.2f}'' "

# Convertir DEC a formato de grados, minutos y segundos
#DEC_gms = grados_a_gms(DEC)
####################################################

#####################################   MÁSCARA PARA NÚCLEO    ##################################################
# Definir el radio de interés en píxeles para el núcleo
R_nucleo = 850  # Radio en píxeles

# Obtener el centro de la imagen
center_x, center_y = height // 2, width // 2

# Crear una máscara circular para limitar la búsqueda del núcleo
Y, X = np.ogrid[:height, :width]
mask_circle_nucleo = (X - center_x) ** 2 + (Y - center_y) ** 2 <= R_nucleo ** 2  # Ecuación del círculo

# Aplicar la máscara circular a la imagen original
masked_M57_nucleo = np.where(mask_circle_nucleo, M57, 0)

# Aplicar umbral de brillo SOLO dentro del círculo del núcleo
umbral_nucleo = np.percentile(masked_M57_nucleo[mask_circle_nucleo], 30)
mask_nucleo = masked_M57_nucleo > umbral_nucleo

# Encontrar las regiones conectadas del núcleo
labeled, num_features = label(mask_nucleo)
region_sizes = np.bincount(labeled.ravel())[1:]  # Contar tamaños de regiones
largest_region_idx = np.argmax(region_sizes) + 1  # Índice de la región más grande
mask_nucleo = labeled == largest_region_idx  # Filtrar solo la región central

# Obtener el centroide ponderado (núcleo de la nebulosa)
centroid = center_of_mass(masked_M57_nucleo * mask_nucleo)
centroid = (int(centroid[0]), int(centroid[1]))  # Convertir a enteros
print(f"Centro refinado de NGC6720: {centroid}")

# Convertir a coordenadas celestes
RA_centroid, DEC_centroid = wcs.all_pix2world([[centroid[1], centroid[0]]], 1)[0]
RA_centroid_hms = grados_a_hms(RA_centroid)
DEC_centroid_gms = grados_a_gms(DEC_centroid)
print(f"Centro refinado de M57 en coordendas (α,δ): ({RA_centroid_hms}, {DEC_centroid_gms}) ")
print('----------------------------------------------------------------------------')

#####################################   MÁSCARA PARA ENVERGADURA    ##################################################
# Definir un radio de interés en píxeles para delimitar la envergadura
R_envergadura = 2170.55  # Ajustar este valor para cubrir solo el anillo

# Crear una máscara circular para limitar la búsqueda de la envergadura
mask_circle_envergadura = (X - centroid[1]) ** 2 + (Y - centroid[0]) ** 2 <= R_envergadura ** 2

# Aplicar umbral de brillo más alto para detectar SOLO la nebulosa dentro de la máscara circular
umbral_envergadura = np.percentile(M57[mask_circle_envergadura], 82)  # Ajuste fino para delimitar el anillo brillante
nebula_mask = np.where(mask_circle_envergadura, M57, 0) > umbral_envergadura  # Aplicar máscara circular

# Determinar los extremos de la línea de envergadura dentro de la máscara de brillo
row = centroid[0]  # Tomamos la fila correspondiente al núcleo detectado
valid_pixels = np.where(nebula_mask[row, :])[0]  # Detectar los píxeles dentro de la nebulosa en esa fila

# Establecer la envergadura SIMÉTRICA respecto al núcleo
max_extent = min(centroid[1] - valid_pixels[0], valid_pixels[-1] - centroid[1])  # Simetría
x_min = int(centroid[1] - max_extent)
x_max = int(centroid[1] + max_extent)

# Convertir a coordenadas ecuatoriales para el plot
RA_start, DEC_start = wcs.all_pix2world([[x_min, row]], 1)[0]
RA_end, DEC_end = wcs.all_pix2world([[x_max, row]], 1)[0]

# -----------------------------------------------------------
# Calcular dirección perpendicular a la envergadura
dir_enverg = np.array([0, 1])  # la envergadura está horizontal → dirección (y=0, x=1)
dir_perp = np.array([1, 0])   # perpendicular → dirección (y=1, x=0)

# Longitud estimada del eje menor (ajusta si quieres)
length_minor = max_extent * 0.75  # por ejemplo, un 75% del eje mayor

# Coordenadas del eje menor (centrado en el núcleo)
x1_minor = centroid[1]
y1_minor = centroid[0] - length_minor
x2_minor = centroid[1]
y2_minor = centroid[0] + length_minor


#####################################   DISTANCIAS   ##################################################
# Definir la envergadura en píxeles (ajustar al valor correcto obtenido)
envergadura_pixeles = 1986.0

# Parámetros conocidos
pixel_size_arcsec = 0.03962000086903572  # Tamaño del píxel en arcsec
distance_ly = 2283  # Distancia a NGC6720 en años luz
arcsec_to_radian = np.pi / (180 * 3600)  # Conversión de arcsec a radianes

# Factores de conversión
pixeles_a_km = 2.6458e-07  # 1 píxel = 2.6458E-07 km
km_a_ly = 1.0570e-13  # 1 km = 1.0570e-13 años luz
km_a_pc = 3.2408e-14  # 1 km = 3.2408e-14 parsec
ly_a_km = 9.4607e12  # Conversión de años luz a kilómetros

# Paso 1: Convertir envergadura a arcsec
envergadura_arcsec = envergadura_pixeles * pixel_size_arcsec

# Paso 2: Convertir arcsec a radianes
theta_radianes = envergadura_arcsec * arcsec_to_radian

# Paso 3: Calcular el tamaño real de la nebulosa en años luz
envergadura_ly = distance_ly * theta_radianes  # Tamaño en años luz
envergadura_km = envergadura_ly * ly_a_km # Tamaño en km
envergadura_pc = envergadura_ly / km_a_pc # Convertir a parsecs

# Mostrar resultados
#print(f"Envergadura de la nebulosa:")
#print(f"- {envergadura_pixeles:.2f} píxeles")
#print(f"Tamaño real de la nebulosa en años luz: {envergadura_ly:.2f} ly")
#print(f"Tamaño real de la nebulosa en kilómetros: {envergadura_km:.2e} km")
#print(f"Tamaño real de la nebulosa en parsecs: {envergadura_pc:.2f} pc")
print('----------------------------------------------------------------------------')

#####################################   TRANSFORMACIÓN DE COORDENADAS   ##################################
# Datos de entrada
Xr = 2097.0
Yr = 2046.0
V2r = 0.4314  # en arcosegundos
V3r = -3.7840 # en arcosegundos
Sx = 0.039750
Sy = 0.039591
beta_x = -41.4701  # en grados
beta_y = 44.7597  # en grados
 # Centro de la imagen, punto a transformar
X = 2193.0 
Y = 2170.0
PA_V3 = 289.999298  # Ángulo PA_V3 en grados

# Conversión de grados a radianes, para poder trabajar con ellos
beta_x_rad = math.radians(beta_x)
beta_y_rad = math.radians(beta_y)
# Convertir PA_V3 a radianes
PA_V3_rad = math.radians(PA_V3)

# Cálculo de V2 arcosegundos
V2 = V2r + Sx * math.sin(beta_x_rad) * (X - Xr) + Sy * math.sin(beta_y_rad) * (Y - Yr)

# Cálculo de V3 arcosegundos
V3 = V3r + Sx * math.cos(beta_x_rad) * (X - Xr) + Sy * math.cos(beta_y_rad) * (Y - Yr)

# Rotación de las coordenadas V2 y V3 al sistema Norte y Este
N = V3 * math.cos(PA_V3_rad) + V2 * math.sin(PA_V3_rad)  # Coordenada Norte relativa
E = V3 * math.sin(PA_V3_rad) + V2 * math.cos(PA_V3_rad)  # Coordenada Este relativa

print('')

#####################################   DETECCIÓN DE ESTRELLAS BRILLANTES   #####################################
# Paso 1: Normalización (si no la tienes ya hecha)
img_norm = (M57 - np.min(M57)) / (np.max(M57) - np.min(M57))

# Paso 2: Detección de picos locales (estrellas brillantes)
coords_estrellas = peak_local_max(img_norm, min_distance=5, threshold_abs=0.4)  # Puedes ajustar el umbral

# Mensaje informativo
print(f"Se han detectado y marcado las {len(coords_estrellas)} estrellas más brillantes.")
####################################   DISTANCIAS FÍSICAS A ESTRELLAS BRILLANTES  #####################################

# Distancia en años luz
d_ly = 2300 # información sacada de internet
ly_to_pc = 0.3066
d_pc = d_ly * ly_to_pc  

# Escala de píxel (arcsec/pix)
pixel_scales = np.abs(wcs.pixel_scale_matrix.diagonal()) * 3600
arcsec_per_pix = np.mean(pixel_scales)

# Calcular distancias en píxeles desde el centroide
pix_distances = [np.linalg.norm(np.array([y, x]) - np.array(centroid)) for y, x in coords_estrellas]

# Pasar a arcsec
arcsec_distances = np.array(pix_distances) * arcsec_per_pix

# Pasar a radianes
theta_rad = (arcsec_distances / 3600) * (np.pi / 180)

# Distancias físicas en parsec
pc_distances = theta_rad * d_pc

# Mostrar 5 resultados más representativos (puedes ajustar el número)
print('\nDistancias desde el núcleo a las estrellas más brillantes:')
for i, (p, a, pc) in enumerate(zip(pix_distances[:5], arcsec_distances[:5], pc_distances[:5]), 1):
    print(f"Estrella {i}: {p:.1f} px ≈ {a:.2f}\" ≈ {pc:.2f} pc")

#####################################   VISUALIZACIÓN   #######################################

# Crear la figura sin distorsiones
fig = plt.figure(figsize=(9, 9))
ax = plt.subplot(projection=wcs)  ## Trabajar en 2D

# Mostrar la imagen con mejor contraste
im = ax.imshow(M57, cmap='inferno', norm=LogNorm(vmin=1e-3, vmax=5e-1))
ax.set_title('M57 - Nebulosa del Anillo (F656N)', fontproperties=tnr_font, fontsize=16, pad=15)
ax.set_xlabel('α', fontproperties=tnr_font, fontsize=12)
ax.set_ylabel('δ', fontproperties=tnr_font, fontsize=12)


# Graficar la línea de envergadura ajustada con máscara circular y SIMÉTRICA
ax.plot([x_min, x_max], [row, row], color='green', linewidth=2, label="Envergadura de la nebulosa")
# Etiqueta para el eje mayor (envergadura)
ax.text((x_min + x_max) / 2 + 900, row + 50, 'Eje mayor',
        color='green', fontsize=13, fontproperties=tnr_font)

# Dibujar eje menor (perpendicular a la envergadura)
ax.plot([x1_minor, x2_minor], [y1_minor, y2_minor], color='blue', linewidth=2, label="Eje menor")
ax.text(centroid[1] + 50, centroid[0] + length_minor + 30, 'Eje menor',
        color='blue', fontsize=13, fontproperties=tnr_font)

# Marcar el centro refinado en la imagen
ax.plot(centroid[1], centroid[0], '+', color='black', markersize=8, label="Centro Refinado")

# Dibujar círculos estrellas más brillantes y líneas desde el núcleo
for i, (y, x) in enumerate(coords_estrellas[:5]):  # solo las 5 más cercanas por claridad
    ax.add_patch(Circle((x, y), radius=10, edgecolor='red', facecolor='none', linewidth=1.2))
    ax.plot([centroid[1], x], [centroid[0], y], color='orange', linestyle='--', linewidth=1.2)

# Anotación con coordenadas
text_str = f"Coordenadas del núcleo\nα: {grados_a_hms(RA_centroid)}\nδ: {grados_a_gms(DEC_centroid)}"
ax.text(0.95, 0.95, text_str, transform=ax.transAxes, color='white', fontsize=10,
        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# Añadir leyenda para N y E
#text_str = f"Nc: {N:.6f} (arcsec)\nEc: {E:.6f} (arcsec)"
#ax.text(0.95, 0.855, text_str, transform=ax.transAxes, color='white', fontsize=10,
#        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# --------------------------
# Añadir Norte y Este en la retícula correcta
# Punto en la esquina inferior derecha para la orientación
x, y = 3750, 520
ra_dec_corner = wcs.pixel_to_world(x, y)

# Desplazamientos para las flechas
delta_ra = (0.002 * u.deg)
delta_dec = (0.002 * u.deg)

# Calcular coordenadas de Norte y Este
ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)

# Convertir de vuelta a píxeles
norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
este_x, este_y = wcs.world_to_pixel(ra_dec_este)

# Dibujar flechas en la dirección correcta
ax.arrow(x, y, norte_x - x, norte_y - y, 
         color='white', head_width=20, head_length=25, label="Norte")
ax.arrow(x, y, este_x - x, este_y - y, 
         color='white', head_width=20, head_length=25, label="Este")

# Etiquetas
ax.text(norte_x + 45, norte_y, 'N', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')
ax.text(este_x, este_y - 140, 'E', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')

# Añadir la barra de color
cbar = plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Brillo', fontproperties=tnr_font, fontsize=12)
cbar.ax.tick_params(labelsize=10)
for label in cbar.ax.get_yticklabels():
    label.set_fontproperties(tnr_font)

#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon = ax.coords[0]  # Eje de Ascensión Recta (α)
lat = ax.coords[1]  # Eje de Declinación (δ)


# Forzar la visualización completa de horas, minutos y segundos en α
lon.set_major_formatter('hh:mm:ss.s')

# Forzar la visualización completa de grados, minutos y segundos en δ
lat.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon.display_minor_ticks(True)
lat.display_minor_ticks(True)
#############################################################################


# Configurar la retícula correctamente
ax.grid(color='white', linestyle='--', linewidth=0.5)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Ajuste del espacio

plt.show()
NGC6720.close()


### Base de código para realizar la composición RGB

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
import matplotlib.font_manager as fm

# Fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar imágenes y WCS (asegurando abrir todos desde la misma carpeta)
F658N_hdu = fits.open('NGC6720/ibh601060_drz.fits')  # Hα
F502N_hdu = fits.open('NGC6720/ibh601070_drz.fits')  # [O III]
F469N_hdu = fits.open('NGC6720/ibh6010c0_drz.fits')  # He II

F658N = np.nan_to_num(F658N_hdu[1].data)
F502N = np.nan_to_num(F502N_hdu[1].data)
F469N = np.nan_to_num(F469N_hdu[1].data)
wcs = WCS(F656N_hdu[1].header)  # Usamos WCS de la primera imagen, por ejemplo

# Igualar tamaños
min_shape = (min(F658N.shape[0], F502N.shape[0], F469N.shape[0]), 
             min(F658N.shape[1], F502N.shape[1], F469N.shape[1]))

F658N_crop = F658N[:min_shape[0], :min_shape[1]]
F502N_crop = F502N[:min_shape[0], :min_shape[1]]
F469N_crop = F469N[:min_shape[0], :min_shape[1]]

# Normalización individual
def normalize_img(img):
    img_clip = np.clip(img, np.percentile(img[img > 0], 5), np.percentile(img[img > 0], 99))
    norm = (img_clip - img_clip.min()) / (img_clip.max() - img_clip.min())
    return norm

R = normalize_img(F658N_crop)
G = normalize_img(F502N_crop)
B = normalize_img(F469N_crop)

# Crear imagen RGB
rgb_image = np.zeros((R.shape[0], R.shape[1], 3))
rgb_image[:, :, 0] = R
rgb_image[:, :, 1] = G
rgb_image[:, :, 2] = B

# Función para flechas N y E
def N_E_arrows(ax, wcs):
    x, y = 3500, 600
    ra_dec_corner = wcs.pixel_to_world(x, y)
    delta_ra = 0.003 * u.deg
    delta_dec = 0.003 * u.deg
    ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
    ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)
    norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
    este_x, este_y = wcs.world_to_pixel(ra_dec_este)

    ax.arrow(x, y, norte_x - x, norte_y - y, color='gray', head_width=40, head_length=35)
    ax.arrow(x, y, este_x - x, este_y - y, color='gray', head_width=40, head_length=35)
    ax.text(norte_x + 45, norte_y + 30, 'N', color='gray', fontsize=12, fontproperties=tnr_font, ha='center')
    ax.text(este_x - 10, este_y - 150, 'E', color='gray', fontsize=12, fontproperties=tnr_font, ha='center')

# Mostrar subplots con cada filtro y la combinación
fig = plt.figure(figsize=(20, 15))
plt.subplots_adjust(wspace=0.4)
# Hα
ax1 = fig.add_subplot(1, 4, 1, projection=wcs)
ax1.imshow(R, cmap='Reds')
ax1.set_title("F658N ([N II])", fontproperties=tnr_font, fontsize=18)
ax1.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax1, wcs)
ax1.set_xlabel('α', fontproperties=tnr_font, fontsize=12)
ax1.set_ylabel('δ', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax1.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax1.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon1 = ax1.coords[0]  # Eje de Ascensión Recta (α)
lat1 = ax1.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon1.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat1.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon1.display_minor_ticks(True)
lat1.display_minor_ticks(True)
#############################################################################



# [O III]
ax2 = fig.add_subplot(1, 4, 2, projection=wcs)
ax2.imshow(G, cmap='Greens')
ax2.set_title("F502N ([O III])", fontproperties=tnr_font, fontsize=18)
ax2.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax2, wcs)
ax2.set_xlabel('α', fontproperties=tnr_font, fontsize=12)
ax2.set_ylabel('δ', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax2.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax2.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon2 = ax2.coords[0]  # Eje de Ascensión Recta (α)
lat2 = ax2.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon2.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat2.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon2.display_minor_ticks(True)
lat2.display_minor_ticks(True)
#############################################################################


# He II
ax3 = fig.add_subplot(1, 4, 3, projection=wcs)
ax3.imshow(B, cmap='Blues')
ax3.set_title("F469N ([He II])", fontproperties=tnr_font, fontsize=18)
ax3.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax3, wcs)
ax3.set_xlabel('α', fontproperties=tnr_font, fontsize=12)
ax3.set_ylabel('δ', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax3.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax3.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon3 = ax3.coords[0]  # Eje de Ascensión Recta (α)
lat3 = ax3.coords[1]  # Eje de Declinación (δ)


# Forzar la visualización completa de horas, minutos y segundos en α
lon3.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat3.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon3.display_minor_ticks(True)
lat3.display_minor_ticks(True)
#############################################################################


# RGB combinado
ax4 = fig.add_subplot(1, 4, 4, projection=wcs)
ax4.imshow(rgb_image)
ax4.set_title("M57 - Composición RGB (A)", fontproperties=tnr_font, fontsize=18)
ax4.grid(color='white', linestyle='--', linewidth=0.5)
N_E_arrows(ax4, wcs)
ax4.set_xlabel('α', fontproperties=tnr_font, fontsize=12)
ax4.set_ylabel('δ', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax4.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax4.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon4 = ax4.coords[0]  # Eje de Ascensión Recta (α)
lat4 = ax4.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon4.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat4.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon4.display_minor_ticks(True)
lat4.display_minor_ticks(True)
#############################################################################


plt.tight_layout()
plt.show()


### Base de código para realizar los perfiles de brillo

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

# Cargar los filtros RGB de la composición A
F658N = np.nan_to_num(fits.getdata('NGC6720/ibh601060_drz.fits'))  # R
F502N = np.nan_to_num(fits.getdata('NGC6720/ibh601070_drz.fits'))  # G
F469N = np.nan_to_num(fits.getdata('NGC6720/ibh6010c0_drz.fits'))  # B

# Igualar tamaño
min_shape = (min(F658N.shape[0], F502N.shape[0], F469N.shape[0]),
             min(F658N.shape[1], F502N.shape[1], F469N.shape[1]))
F658N = F658N[:min_shape[0], :min_shape[1]]
F502N = F502N[:min_shape[0], :min_shape[1]]
F469N = F469N[:min_shape[0], :min_shape[1]]

# Centro
center_x, center_y = min_shape[0] // 2, min_shape[1] // 2

# Suavizado opcional (mejor para perfiles suaves)
F658N_smooth = gaussian_filter(F658N, sigma=1.5)
F502N_smooth = gaussian_filter(F502N, sigma=1.5)
F469N_smooth = gaussian_filter(F469N, sigma=1.5)

# Calcular perfil radial
def perfil_radial(imagen, centro, r_max):
    y, x = np.indices(imagen.shape)
    r = np.sqrt((x - centro[1])**2 + (y - centro[0])**2).astype(int)
    perfil = np.zeros(r_max)
    for i in range(r_max):
        mask = (r == i)
        if np.sum(mask) > 0:
            perfil[i] = np.mean(imagen[mask])
    return perfil

# Radios hasta los que analizar (ajusta si quieres más lejos)
r_max = 100

# Calcular perfiles para R, G, B
perfil_R = perfil_radial(F658N_smooth, (center_x, center_y), r_max)
perfil_G = perfil_radial(F502N_smooth, (center_x, center_y), r_max)
perfil_B = perfil_radial(F469N_smooth, (center_x, center_y), r_max)

# --- PLOT ---
plt.figure(figsize=(8,6))
plt.plot(perfil_R, color='red', label='[N II] - F658N (R)')
plt.plot(perfil_G, color='green', label='[O III] - F502N (G)')
plt.plot(perfil_B, color='blue', label='He II - F469N (B)')
plt.title("Perfil Radial - Composición RGB A (M57)")
plt.xlabel("Distancia radial (pixeles)")
plt.ylabel("Brillo promedio")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## **M42 - Nebulosa de Orión**

Base de código para el cálculo de: <br>
- centroide (MÁSCARA PARA NÚCLEO)
- eje mayor y menor (EJE PRINCIPAL DE LA NEBULOSA)
- transformación XY - V2V3 - NE (TRANSFORMACIÓN DE COORDENADAS)
- estrellas brillantes (DETECCIÓN DE ESTRELLAS BRILLANTES)
- visualización con todo lo anterior, orientación, retícula, ejes... (VISUALIZACIÓN)


In [ ]:
# Librerías necesarias
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.colors import LogNorm
from astropy.wcs import WCS
import math
from astropy import units as u
from astropy.coordinates import SkyCoord
from scipy.ndimage import label, center_of_mass
from skimage.measure import regionprops
from scipy.ndimage import gaussian_filter
from matplotlib.patches import Circle
from skimage.feature import peak_local_max

# Cargar la fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar los datos de la imagen
F212N = fits.open('M42_JWST_NIRCAM/jw01256-o001_t001_nircam_clear-f212n_i2d.fits')  # FITS file
F212N_h = F212N[0].header
F212N_d = fits.getdata('M42_JWST_NIRCAM/jw01256-o001_t001_nircam_clear-f212n_i2d.fits')
F212N_d = np.nan_to_num(F212N_d)  # Reemplazar NaN por 0

# Obtener dimensiones
height, width = F212N_d.shape
print(f"Píxeles de la imagen: {F212N_d.shape}")
print('----------------------------------------------------------------------------')
# Obtener WCS
wcs = WCS(F212N[1].header, naxis=2)

###################### PASO DE AR - h m s Y DEC º ' " ######################
# Función para convertir grados a horas, minutos, segundos (1h son 15º)
def grados_a_hms(ar_deg):
    horas = int(ar_deg / 15)  # Convertir grados a horas
    minutos = int((ar_deg / 15 - horas) * 60)  # Convertir la parte fraccionaria de horas a minutos
    segundos = ((ar_deg / 15 - horas) * 60 - minutos) * 60  # Convertir la parte fraccionaria de minutos a segundos
    return f"{horas}h {minutos}m {segundos:.2f}s"
# Convertir AR a formato de horas, minutos y segundos
#AR_hms = grados_a_hms(AR)

# Función para convertir grados a grados, minutos y segundos
def grados_a_gms(deg):
    grados = int(deg)  # Parte entera de los grados
    minutos = int(abs(deg - grados) * 60)  # Convertir la parte decimal de grados a minutos
    segundos = (abs(deg - grados) * 60 - minutos) * 60  # Convertir la parte decimal de minutos a segundos
    return f"{grados}° {minutos}' {segundos:.2f}'' "

# Convertir DEC a formato de grados, minutos y segundos
#DEC_gms = grados_a_gms(DEC)
####################################################

#####################################   MÁSCARA PARA NÚCLEO    ##################################################
# Definir el radio de interés en píxeles para el núcleo
R_nucleo = 6500  # Radio en píxeles

# Obtener el centro de la imagen
center_x, center_y = height // 2, width // 2

# Crear una máscara circular para limitar la búsqueda del núcleo
Y, X = np.ogrid[:height, :width]
mask_circle_nucleo = (X - center_x) ** 2 + (Y - center_y) ** 2 <= R_nucleo ** 2  # Ecuación del círculo

# Aplicar la máscara circular a la imagen original
masked_F212N_nucleo = np.where(mask_circle_nucleo, F212N_d, 0)

# Aplicar umbral de brillo SOLO dentro del círculo del núcleo
umbral_nucleo = np.percentile(masked_F212N_nucleo[mask_circle_nucleo], 98)
mask_nucleo = masked_F212N_nucleo > umbral_nucleo

# Encontrar las regiones conectadas del núcleo
labeled, num_features = label(mask_nucleo)
region_sizes = np.bincount(labeled.ravel())[1:]  # Contar tamaños de regiones
largest_region_idx = np.argmax(region_sizes) + 1  # Índice de la región más grande
mask_nucleo = labeled == largest_region_idx  # Filtrar solo la región central

# Obtener el centroide ponderado (núcleo de la nebulosa)
centroid = center_of_mass(masked_F212N_nucleo * mask_nucleo)
centroid = (int(centroid[0]), int(centroid[1]))  # Convertir a enteros
print(f"Centro refinado de M42: {centroid}")

# Convertir a coordenadas celestes
RA_centroid, DEC_centroid = wcs.all_pix2world([[centroid[1], centroid[0]]], 1)[0]
RA_centroid_hms = grados_a_hms(RA_centroid)
DEC_centroid_gms = grados_a_gms(DEC_centroid)
print(f"Centro refinado de M42 en coordendas (α,δ): ({RA_centroid_hms}, {DEC_centroid_gms}) ")
print('----------------------------------------------------------------------------')

#####################################   TRANSFORMACIÓN DE COORDENADAS   ##################################
# Calcular escala de píxel (en grados/píxel → luego se convierte a arcsec/píxel)
pixel_scales_deg = wcs.pixel_scale_matrix.diagonal()
pixel_scale = np.abs(pixel_scales_deg) * 3600  # Conversión a arcsec

# Datos de entrada
# Sabiendo que APERNAME= 'NRCA1_FULL' usamos los valores de A1
Xr = -122.49
Yr = -34.77
V2r = 120.58  # en arcosegundos
V3r = -527.50 # en arcosegundos

Sx, Sy = pixel_scale[0], pixel_scale[1]

beta_x = 0  # en grados
beta_y = 90  # en grados
 # Centroide de la imagen, punto a transformar
X = 10442.0 
Y = 5880.0
PA_V3 = 270.0736538196815  # Ángulo PA_V3 en grados

# Conversión de grados a radianes, para poder trabajar con ellos
beta_x_rad = math.radians(beta_x)
beta_y_rad = math.radians(beta_y)
# Convertir PA_V3 a radianes
PA_V3_rad = math.radians(PA_V3)

# Cálculo de V2 arcosegundos
V2 = V2r + Sx * math.sin(beta_x_rad) * (X - Xr) + Sy * math.sin(beta_y_rad) * (Y - Yr)

# Cálculo de V3 arcosegundos
V3 = V3r + Sx * math.cos(beta_x_rad) * (X - Xr) + Sy * math.cos(beta_y_rad) * (Y - Yr)

# Rotación de las coordenadas V2 y V3 al sistema Norte y Este
N = V3 * math.cos(PA_V3_rad) + V2 * math.sin(PA_V3_rad)  # Coordenada Norte relativa
E = V3 * math.sin(PA_V3_rad) + V2 * math.cos(PA_V3_rad)  # Coordenada Este relativa


# Conversión a arcmin (1 arcmin = 60 arcsec)
N_arcmin = N / 60
E_arcmin = E / 60

# Mostrar los resultados
print(f"Coordenada Norte: {N:.2f} arcsec → {N_arcmin:.2f} arcmin")
print(f"Coordenada Este : {E:.2f} arcsec → {E_arcmin:.2f} arcmin")
print('')

#####################################   VISUALIZACIÓN   #######################################

# Crear la figura sin distorsiones
fig = plt.figure(figsize=(8, 8))
ax = plt.subplot(projection=wcs)  ## Trabajar en 2D

# Mostrar la imagen con mejor contraste
im = ax.imshow(F212N_d, cmap='inferno', norm=LogNorm(vmin=vmin, vmax=vmax))
# --- Dibujar contornos que resalten las regiones ---
#ax.contour(F212N_smooth, levels=levels, colors='cyan', linewidths=0.8, alpha=0.7)

ax.set_title('M42 - Nebulosa de Orión (F212N)', fontproperties=tnr_font, fontsize=16, pad=15)
ax.set_xlabel('δ', fontproperties=tnr_font, fontsize=12)
ax.set_ylabel('α', fontproperties=tnr_font, fontsize=12)
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax.coords[0].set_ticklabel(size=12, fontproperties=tnr_font)  # RA (α)
ax.coords[1].set_ticklabel(size=12, fontproperties=tnr_font)  # DEC (δ)


# Marcar el centro refinado en la imagen
ax.plot(centroid[1], centroid[0], '+', color='black', markersize=8, label="Centro Refinado")

# Anotación con coordenadas
text_str = f"Coordenadas del núcleo\nα: {grados_a_hms(RA_centroid)}\nδ: {grados_a_gms(DEC_centroid)}"
ax.text(0.93, 0.95, text_str, transform=ax.transAxes, color='white', fontsize=10,
        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# Añadir leyenda para N y E
#text_str = f"Nc: {N:.6f} (arcsec)\nEc: {E:.6f} (arcsec)"
#ax.text(0.95, 0.855, text_str, transform=ax.transAxes, color='white', fontsize=10,
#        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# --------------------------
# Añadir Norte y Este en la retícula correcta
# Punto en la esquina inferior derecha para la orientación
x, y = 13106.25, 3669.25
ra_dec_corner = wcs.pixel_to_world(x, y)

# Desplazamientos para las flechas
delta_ra = (0.01 * u.deg)
delta_dec = (0.01 * u.deg)

# Calcular coordenadas de Norte y Este
ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)

# Convertir de vuelta a píxeles
norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
este_x, este_y = wcs.world_to_pixel(ra_dec_este)

# Dibujar flechas en la dirección correcta
ax.arrow(x, y, norte_x - x, norte_y - y, 
         color='white', head_width=70, head_length=80, label="Norte")
ax.arrow(x, y, este_x - x, este_y - y, 
         color='white', head_width=70, head_length=80, label="Este")

# Etiquetas
ax.text(norte_x - 100, norte_y, 'N', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')
ax.text(este_x + 200, este_y - 490, 'E', color='white', fontsize=13, fontproperties=tnr_font, ha='center', va='bottom')

# Añadir la barra de color
cbar = plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Brillo', fontproperties=tnr_font, fontsize=12)
cbar.ax.tick_params(labelsize=10)
for label in cbar.ax.get_yticklabels():
    label.set_fontproperties(tnr_font)

#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon = ax.coords[0]  # Eje de Ascensión Recta (α)
lat = ax.coords[1]  # Eje de Declinación (δ)


# Forzar la visualización completa de horas, minutos y segundos en α
lon.set_major_formatter('hh:mm:ss.s')

# Forzar la visualización completa de grados, minutos y segundos en δ
lat.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon.display_minor_ticks(True)
lat.display_minor_ticks(True)
#############################################################################


# Configurar la retícula correctamente
ax.grid(color='white', linestyle='--', linewidth=0.5)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Ajuste del espacio

plt.show()
F212N.close()


### Base de código para realizar el recorte y zoom de la zona central de la imagen

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from scipy.ndimage import gaussian_filter
from skimage.feature import peak_local_max
from matplotlib.patches import Circle
from astropy.nddata import Cutout2D

# Cargar los datos de la imagen
F212N = fits.open('M42_JWST_NIRCAM/jw01256-o001_t001_nircam_clear-f212n_i2d.fits')  # FITS file
F212N_h = F212N[0].header
F212N_d = fits.getdata('M42_JWST_NIRCAM/jw01256-o001_t001_nircam_clear-f212n_i2d.fits')
F212N_d = np.nan_to_num(F212N_d)  # Reemplazar NaN por 0
# Obtener WCS
wcs = WCS(F212N[1].header, naxis=2)

# Obtener el centroide ponderado (núcleo de la nebulosa)
centroid = center_of_mass(masked_F212N_nucleo * mask_nucleo)
centroid = (int(centroid[0]), int(centroid[1]))  # Convertir a enteros

# Define el tamaño del recorte alrededor del núcleo
recorte_size = 5000  # píxeles
x_center, y_center = centroid[1], centroid[0]
x_min = int(x_center - recorte_size / 2)
x_max = int(x_center + recorte_size / 2)
y_min = int(y_center - recorte_size / 2)
y_max = int(y_center + recorte_size / 2)

# Extrae el recorte del núcleo
#recorte_nucleo = F212N_d[y_min:y_max, x_min:x_max]

# Crear el cutout con WCS
position = (x_center, y_center)
size = (recorte_size, recorte_size)
cutout = Cutout2D(F212N_d, position, size, wcs=wcs)
recorte_nucleo = cutout.data
wcs_recorte = cutout.wcs

# Aplica un filtro gaussiano para suavizar la imagen
recorte_suavizado = gaussian_filter(recorte_nucleo, sigma=3)

#####################################################################################################
# Normalizar el recorte
recorte_norm = (recorte_nucleo - np.min(recorte_nucleo)) / (np.max(recorte_nucleo) - np.min(recorte_nucleo))

# Detectar estrellas brillantes solo en el recorte
coords_estrellas_recorte = peak_local_max(recorte_norm, min_distance=5, threshold_abs=0.92)

print(f"Se han detectado {len(coords_estrellas_recorte)} estrellas brillantes en el recorte.")

#################################################################
# Define niveles de contorno
vmin = np.percentile(F212N_d[F212N_d > 0], 5)
vmax = np.percentile(F212N_d[F212N_d > 0], 99)
levels = np.logspace(np.log10(vmin), np.log10(vmax), num=6)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Imagen completa con recorte marcado
ax1 = fig.add_subplot(1, 2, 1, projection=wcs)
ax1.imshow(F212N_d, cmap='inferno', norm=LogNorm(vmin=vmin, vmax=vmax))
ax1.add_patch(Rectangle((x_min, y_min), recorte_size, recorte_size,
                        edgecolor='cyan', facecolor='none', linewidth=2))
# Marcar el centro refinado en la imagen
ax1.plot(centroid[1], centroid[0], '+', color='black', markersize=8, label="Centro Refinado")
ax1.set_title('M42 (F212N)- Imagen Completa')
ax1.axis('off')

# Imagen del recorte con estrellas brillantes detectadas
ax2 = fig.add_subplot(1, 2, 2, projection=wcs_recorte)
ax2.imshow(recorte_nucleo, cmap='inferno', norm=LogNorm(vmin=vmin, vmax=vmax))
for i, (y, x) in enumerate(coords_estrellas_recorte):
    ax2.add_patch(Circle((x, y), radius=30, edgecolor='red', facecolor='none', linewidth=1.0))
    ax2.contour(recorte_suavizado, levels=levels, colors='gray', linewidths=0.5)
# Marcar el centro refinado en la imagen
#ax2.plot(centroid[1], centroid[0], '+', color='black', markersize=8, label="Centro Refinado")    
ax2.set_title('Núcleo de M42')
ax2.axis('off')

plt.tight_layout()
plt.show()



### Base de código para realizar la composición RGB

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
import matplotlib.font_manager as fm

# Fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar imágenes y WCS (asegurando abrir todos desde la misma carpeta)
F444W_hdu = fits.open('M42_JWST_NIRCAM/jw01256-o001_t001_nircam_f444w-f470n_i2d.fits') 
F277W_hdu = fits.open('M42_JWST_NIRCAM/jw01256-o001_t001_nircam_clear-f277w_i2d.fits')  
F150W2_hdu = fits.open('M42_JWST_NIRCAM/jw01256-o001_t001_nircam_f150w2-f162m_i2d.fits')  

F444W = np.nan_to_num(F444W_hdu[1].data)
F277W = np.nan_to_num(F277W_hdu[1].data)
F150W2 = np.nan_to_num(F150W2_hdu[1].data)
wcs = WCS(F444W_hdu[1].header)  # Usamos WCS de la primera imagen, por ejemplo

# Igualar tamaños
min_shape = (min(F444W.shape[0], F277W.shape[0], F150W2.shape[0]), 
             min(F444W.shape[1], F277W.shape[1], F150W2.shape[1]))

F444W_crop = F444W[:min_shape[0], :min_shape[1]]
F277W_crop = F277W[:min_shape[0], :min_shape[1]]
F150W2_crop = F150W2[:min_shape[0], :min_shape[1]]

# Normalización individual
def normalize_img(img):
    img_clip = np.clip(img, np.percentile(img[img > 0], 5), np.percentile(img[img > 0], 99))
    norm = (img_clip - img_clip.min()) / (img_clip.max() - img_clip.min())
    return norm

R = normalize_img(F444W_crop)
G = normalize_img(F277W_crop)
B = normalize_img(F150W2_crop)

# Crear imagen RGB
rgb_image = np.zeros((R.shape[0], R.shape[1], 3))
rgb_image[:, :, 0] = R
rgb_image[:, :, 1] = G
rgb_image[:, :, 2] = B

# Función para flechas N y E
def N_E_arrows(ax, wcs):
    x, y = 6500.25, 1750.25
    ra_dec_corner = wcs.pixel_to_world(x, y)
    delta_ra = 0.01 * u.deg
    delta_dec = 0.01 * u.deg
    ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
    ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)
    norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
    este_x, este_y = wcs.world_to_pixel(ra_dec_este)

    ax.arrow(x, y, norte_x - x, norte_y - y, color='gray', head_width=70, head_length=80)
    ax.arrow(x, y, este_x - x, este_y - y, color='gray', head_width=70, head_length=80)
    ax.text(norte_x - 100, norte_y + 85, 'N', color='gray', fontsize=12, fontproperties=tnr_font, ha='center')
    ax.text(este_x + 150, este_y - 300, 'E', color='gray', fontsize=12, fontproperties=tnr_font, ha='center')

# Mostrar subplots con cada filtro y la combinación
fig = plt.figure(figsize=(22, 9))
plt.subplots_adjust(wspace=0.4)
# Hα
ax1 = fig.add_subplot(1, 4, 1, projection=wcs)
ax1.imshow(R, cmap='Reds')
ax1.set_title("F444W (polvo muy frío)", fontproperties=tnr_font, fontsize=18)
ax1.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax1, wcs)
ax1.set_xlabel('δ', fontproperties=tnr_font, fontsize=12)
ax1.set_ylabel('α', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax1.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax1.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon1 = ax1.coords[0]  # Eje de Ascensión Recta (α)
lat1 = ax1.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon1.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat1.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon1.display_minor_ticks(True)
lat1.display_minor_ticks(True)
#############################################################################



# [O III]
ax2 = fig.add_subplot(1, 4, 2, projection=wcs)
ax2.imshow(G, cmap='Greens')
ax2.set_title("F277W (polvo caliente)", fontproperties=tnr_font, fontsize=18)
ax2.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax2, wcs)
ax2.set_xlabel('δ', fontproperties=tnr_font, fontsize=12)
ax2.set_ylabel('α', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax2.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax2.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon2 = ax2.coords[0]  # Eje de Ascensión Recta (α)
lat2 = ax2.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon2.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat2.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon2.display_minor_ticks(True)
lat2.display_minor_ticks(True)
#############################################################################


# He II
ax3 = fig.add_subplot(1, 4, 3, projection=wcs)
ax3.imshow(B, cmap='Blues')
ax3.set_title("F150W2 (estrellas jóvenes)", fontproperties=tnr_font, fontsize=18)
ax3.grid(color='gray', linestyle='--', linewidth=0.5)
N_E_arrows(ax3, wcs)
ax3.set_xlabel('δ', fontproperties=tnr_font, fontsize=12)
ax3.set_ylabel('α', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax3.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax3.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon3 = ax3.coords[0]  # Eje de Ascensión Recta (α)
lat3 = ax3.coords[1]  # Eje de Declinación (δ)


# Forzar la visualización completa de horas, minutos y segundos en α
lon3.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat3.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon3.display_minor_ticks(True)
lat3.display_minor_ticks(True)
#############################################################################


# RGB combinado
ax4 = fig.add_subplot(1, 4, 4, projection=wcs)
ax4.imshow(rgb_image)
ax4.set_title("M42 - Composición RGB (A)", fontproperties=tnr_font, fontsize=18)
ax4.grid(color='white', linestyle='--', linewidth=0.5)
N_E_arrows(ax4, wcs)
ax4.set_xlabel('δ', fontproperties=tnr_font, fontsize=12)
ax4.set_ylabel('α', fontproperties=tnr_font, fontsize=12)
#############################################################################
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax4.coords[0].set_ticklabel(size=18, fontproperties=tnr_font, simplify=False)  # α (RA)
ax4.coords[1].set_ticklabel(size=18, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Configurar la retícula para mostrar TODAS las unidades completas
lon4 = ax4.coords[0]  # Eje de Ascensión Recta (α)
lat4 = ax4.coords[1]  # Eje de Declinación (δ)

# Forzar la visualización completa de horas, minutos y segundos en α
lon4.set_major_formatter('hh:mm:ss.s')
# Forzar la visualización completa de grados, minutos y segundos en δ
lat4.set_major_formatter('dd:mm:ss')

# Ajustar el espaciado para mejorar la legibilidad
lon4.display_minor_ticks(True)
lat4.display_minor_ticks(True)
#############################################################################


plt.tight_layout()
plt.show()


### Base de código para realizar los perfiles de brillo

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

# Cargar los filtros RGB de la composición A
F444W = np.nan_to_num(fits.getdata('M42_JWST_NIRCAM/jw01256-o001_t001_nircam_f444w-f470n_i2d.fits'))  # R
F277W = np.nan_to_num(fits.getdata('M42_JWST_NIRCAM/jw01256-o001_t001_nircam_clear-f277w_i2d.fits'))  # G
F150W2 = np.nan_to_num(fits.getdata('M42_JWST_NIRCAM/jw01256-o001_t001_nircam_f150w2-f162m_i2d.fits'))  # B

# Igualar tamaño
min_shape = (min(F444W.shape[0], F277W.shape[0], F150W2.shape[0]),
             min(F444W.shape[1], F277W.shape[1], F150W2.shape[1]))
F444W = F444W[:min_shape[0], :min_shape[1]]
F277W = F277W[:min_shape[0], :min_shape[1]]
F150W2 = F150W2[:min_shape[0], :min_shape[1]]

# Centro
center_x, center_y = min_shape[0] // 2, min_shape[1] // 2

# Suavizado opcional (mejor para perfiles suaves)
F444W_smooth = gaussian_filter(F444W, sigma=1.5)
F277W_smooth = gaussian_filter(F277W, sigma=1.5)
F150W2_smooth = gaussian_filter(F150W2, sigma=1.5)

# Calcular perfil radial
def perfil_radial(imagen, centro, r_max):
    y, x = np.indices(imagen.shape)
    r = np.sqrt((x - centro[1])**2 + (y - centro[0])**2).astype(int)
    perfil = np.zeros(r_max)
    for i in range(r_max):
        mask = (r == i)
        if np.sum(mask) > 0:
            perfil[i] = np.mean(imagen[mask])
    return perfil

# Radios hasta los que analizar (ajusta si quieres más lejos)
r_max = 100

# Calcular perfiles para R, G, B
perfil_R = perfil_radial(F444W_smooth, (center_x, center_y), r_max)
perfil_G = perfil_radial(F277W_smooth, (center_x, center_y), r_max)
perfil_B = perfil_radial(F150W2_smooth, (center_x, center_y), r_max)

# --- PLOT ---
plt.figure(figsize=(8,6))
plt.plot(perfil_R, color='red', label='(emisión térmica polvo caliente) - F444W (R)')
plt.plot(perfil_G, color='green', label='(menor emisión térmica polvo caliente) - F277W (G)')
plt.plot(perfil_B, color='blue', label='(emisión general de NIR) - F150W2 (B)')
plt.title("Perfil Radial - Composición RGB A (M42)")
plt.xlabel("Distancia radial (pixeles)")
plt.ylabel("Brillo promedio")
plt.legend(loc='upper right', bbox_to_anchor=(1, 0.85))
#plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
